Paper: City formation by dual migration of firms and workers  
Author: Kensuke Ohtake

In [ ]:
using Random
using Distributions
using Plots
using Dates
using Format
using DataFrames
using CSV

# simulation function
function dualmigration_simulation(pls)

    # random seed
    SD = Dates.millisecond(now()) # current time
    Random.seed!(SD) # set seed
    
    # control parameters 
    sig = pls[1] # elasticity of substitution of manufactured goods
    tau = pls[2] # manufacturing transport cost parameter

    # function of shorter distance on S
    function dist(z)
        minimum([abs(z), 2.0 * π - abs(z)])
    end

    # hyper parameters
    R = 256 # number of space nodes
    dx = 2.0 * π / convert(Float64, R) # size of space element
    dt = 0.01 # size of time step
    mu = 0.6 # manufacturing share
    eps = 1e-10 # acceptable error
    v_m = 1.0 # realwage sensitivity of workers
    v_n = 1.0 # realprofit sensitivity of firms

    # x-coordinate
    x = zeros(Float64, R, 1)
    for i in 1:R
        x[i] = -π + (i - 1) * dx
    end
    
    # agricultural workers (density)
    ph = ones(Float64, (R, 1)) / (2.0 * π)
    
    # transport matrix for manufacturing
    Ex = zeros(Float64, (R, R))
    for i in 1:R
        for j in 1:R
            Ex[i,j] = exp(- tau * (sig - 1.0) * dist(x[i]-x[j]))
        end
    end
    
    # integral operator for instantaneous equilibrium
    function IntegOperat(nominalwage_power, agr_workers, workers, firms)# Instantaneous equilibrium operator
        
        #= 
        Inputs must be array (R, 1)
        nominalwage_power: manufacturing nominal wage raised to the power of sigma
        agr_workers: density of agricultural workers phi(x)
        workers: density of manufactirng workers m(x)
        firms: density of manufacturing firms n(t, x)
        =#

        # nominal income (but sigma omitted)
        nominalincome = (1.0 - mu) * agr_workers + mu * (nominalwage_power .^ (1.0 / sig)) .* workers

        # price index raised to the power of 1-sigma (but dx and sigma omitted)
        priceindex_power = Ex * (firms .* (nominalwage_power .^ ((1.0 - sig) / sig)))

        # output of integral operator (but dx omitted) 
        output = (firms ./ workers) .* (Ex * (nominalincome ./ priceindex_power))
        
        return output
    end
    
    # function to obtain right-hand side of evolution equation
    function RHS(pot_nominalwage_power, agr_workers, workers, firms)
        
        #=
        Inputs must be array (R, 1)
        pot_nominalwage_power: potential solution for instantaneous equilibrium equation
        (manufacturing nominalwage reised to the power of sigma)
        =#
        
        # solve fixed point problem for instantaneous equilibrium by iterative method
        nominalwage_power = copy(pot_nominalwage_power) # initial value for iteration
        while true
            
            old_nominalwage_power = copy(nominalwage_power) # store the previous value
            nominalwage_power = IntegOperat(old_nominalwage_power, agr_workers, workers, firms) # update the value
            
            if maximum(abs.(nominalwage_power - old_nominalwage_power)) < eps # stop condition (max norm < epsilon)
                break
            end
            
        end

        # other equilibrium variables
        priceindex = (sig * dx * Ex * (firms .* (nominalwage_power .^ ((1.0 - sig) / sig)))) .^ (1.0 / (1.0 - sig)) # manufacturing price index
        realwage = (nominalwage_power .^ (1.0 / sig)) .* (priceindex .^ (- mu)) # manufacturing real wage
        realprofit = (mu / sig) * (workers ./ firms) .* realwage # manufacturing real profit

        # # verification that realwage is a column vector
        # if size(realwage) != (R, 1)
        #     error("realwage shape error!")
        # end
        
        # right-hand side of evolution equations
        rhs_workers = v_m * ((realwage .- (transpose(realwage) * workers * dx)) .* workers)
        rhs_firms = v_n * ((realprofit .- (transpose(realprofit) * firms * dx)) .* firms)
        
        return rhs_workers, rhs_firms, nominalwage_power
    end
    
    #=
    simulation of the evolution equations
    =#

    # initial values of manufacturing workers
    delta_m = rand(Uniform(-0.0005, 0.0005), (R, 1)) # perturbation generated by random numbers
    delta_m = delta_m .- (sum(delta_m) * dx) / (2.0 * π) # total integral equals to 0
    m = (ones(Float64, (R, 1)) / (2.0 * π)) + delta_m # manufacturing workers: homogeneous state + perturbation

    # initial values of firms
    delta_n = rand(Uniform(-0.0005, 0.0005), (R, 1)) # perturbation generated by random numbers
    delta_n = delta_n .- (sum(delta_n) * dx) / (2.0 * π) # total equals to 0
    n = (ones(Float64, (R, 1)) / (2.0 * π)) + delta_n # firms: homogeneous state + perturbation

    # verify difference in initial values of m and n
    if n == m
        error("same initial value !")
    end

    # check non-negativity
    if sum(sign.(m)) != R || sum(sign.(n)) != R
        error("non-negativity error!")
    end

    # check conservation
    if 1.0 - 1e-10 > sum(m) * dx || 1.0 - 1e-10 > sum(n) * dx # The total integral is too small
        error("conservation error!")
    elseif sum(m) * dx > 1.0 + 1e-10 || sum(n) * dx > 1.0 + 1e-10 # The total integral is too large
        error("conservation error!")
    end

    # save initial values
    ini_m = copy(m)
    println("initial total manufacturing workers is ", sum(ini_m) * dx)
    ini_n = copy(n)
    println("initial total firms is ", sum(ini_n) * dx)
    
    t = 0 # initial time index
    W = ones(Float64, (R, 1)) # initial potential solution to instantaneous equilibrium equation

    while true

        # store the previous values
        old_m = copy(m) # manufacturing workers
        old_n = copy(n) # manufacturing firms
        old_W = copy(W) # instantaneous equilibrium
        
        # 1-st order Runge Kutta method (Euler method) for the replicator equation 
        k1m, k1n, W = RHS(old_W, ph, m, n) # obtain right-hand sides of evolution equations and instantaneous equilibrium
        dm = dt * k1m # update size of m
        dn = dt * k1n # update size of n
        m = old_m + dm # update m by Euler method
        n = old_n + dn # update n by Euler method

        # check non-negativity
        if sum(sign.(m)) != R || sum(sign.(n)) != R
            println("non-negativity error!")
            break
        end
        
        # check conservation
        if 1.0 - 1e-10 > sum(m) * dx || 1.0 - 1e-10 > sum(n) * dx # The total integral is too small
            println("conservation error!")
            break
        elseif sum(m) * dx > 1.0 + 1e-10 || sum(n) * dx > 1.0 + 1e-10 # The total integral is too large
            println("conservation error!")
            break
        end

        # stop condition (max norm < epsilon)
        if maximum(abs.(dm)) < eps && maximum(abs.(dn)) < eps
            break
        end
    
        t += 1 # update time index
    
    end

    #=
    obtain stationary solution
    =#
    
    # compute instantaneous equilibrium for the final m and n
    stationary_W = copy(W)
    while true
        old_stationary_W = copy(stationary_W) # store the previpus stationary W
        stationary_W = IntegOperat(old_stationary_W, ph, m, n) # update stationary W
        if maximum(abs.(stationary_W - old_stationary_W)) < eps # stop condition (max norm < epsilon)
            break
        end
    end

    # stationary states
    w = stationary_W .^ (1.0 / sig) # nominal wage
    G = (sig * dx * Ex * (n .* (w .^ (1.0 - sig)))) .^ (1.0 / (1.0 - sig)) # price index
    omega = w .* (G .^ (-mu)) # real wage
    eta = (mu / sig) * (m ./ n) .* omega # real profit

    # show total density
    println("Final total manufacturing workers is ", sum(m) * dx)
    println("Final total manufacturing firms is ", sum(n) * dx)

    #=
    output results
    =#
    
    # plot
    DateTime = Dates.format(now(), "yyyymmddHHMMSS")# get current time
    plt_x = append!(reshape(x, R), π) # periodic condition for x-coordinate
    plt_ini_m = append!(reshape(ini_m, R), ini_m[1]) # periodic condition for initial m
    plt_ini_n = append!(reshape(ini_n, R), ini_n[1]) # periodic condition for initial n
    plt_m = append!(reshape(m, R), m[1]) # periodic condition for m
    plt_n = append!(reshape(n, R), n[1]) # periodic condition for n
    plt_w = append!(reshape(w, R), w[1]) # periodic condition for w
    plt_G = append!(reshape(G, R), G[1]) # periodic condition for G
    plt_omega = append!(reshape(omega, R), omega[1]) # periodic condition for omega
    plt_eta = append!(reshape(eta, R), eta[1]) # periodic condition for eta

    filename_initial = format("{}-initial_tau{}sigma{}mu{}R{}t{}sd{}", DateTime, tau, sig, mu, R, t, SD)
    newfilename_initial = replace(filename_initial, "."=>"p")
    filename_m = format("{}-workers_tau{}sigma{}mu{}R{}t{}sd{}", DateTime, tau, sig, mu, R, t, SD)
    newfilename_m = replace(filename_m, "."=>"p")
    filename_n = format("{}-firms_tau{}sigma{}mu{}R{}t{}sd{}", DateTime, tau, sig, mu, R, t, SD)
    newfilename_n = replace(filename_n, "."=>"p")  
    filename_w = format("{}-nominalwage_tau{}sigma{}mu{}R{}t{}sd{}", DateTime, tau, sig, mu, R, t, SD)
    newfilename_w = replace(filename_w, "."=>"p")
    filename_G = format("{}-priceindex_tau{}sigma{}mu{}R{}t{}sd{}", DateTime, tau, sig, mu, R, t, SD)
    newfilename_G = replace(filename_G, "."=>"p") 
    filename_omega = format("{}-realwage_tau{}sigma{}mu{}R{}t{}sd{}", DateTime, tau, sig, mu, R, t, SD)
    newfilename_omega = replace(filename_omega, "."=>"p")
    filename_eta = format("{}-realprofit_tau{}sigma{}mu{}R{}t{}sd{}", DateTime, tau, sig, mu, R, t, SD)
    newfilename_eta = replace(filename_eta, "."=>"p")

    plot(plt_x, plt_ini_m, title="initial values", label="initial worlers", dpi=300)
    plot!(plt_x, plt_ini_n, title="initial values", label="initial firms", dpi=300)
    savefig(newfilename_initial * ".png")
    plot(plt_x, plt_m, title="workers", label="", dpi=300)
    savefig(newfilename_m * ".png")
    plot(plt_x, plt_n, title="firms", label="", dpi=300)
    savefig(newfilename_n * ".png")
    plot(plt_x, plt_w, title="nominal wage", label="", dpi=300)
    savefig(newfilename_w * ".png")
    plot(plt_x, plt_G, title="price index", label="", dpi=300)
    savefig(newfilename_G * ".png")
    plot(plt_x, plt_omega, title="realwage", label="", dpi=300)
    savefig(newfilename_omega * ".png")
    plot(plt_x, plt_eta, title="realprofit", label="", dpi=300)
    savefig(newfilename_eta * ".png")
    
    # save data as csv file
    df = DataFrame(coordinates = plt_x,
                   initial_pop_share_n = plt_ini_n,
                   initial_pop_share_m = plt_ini_m,
                   final_pop_share_n = plt_n,
                   final_pop_share_m = plt_m,
                   manuf_nominal_wage = plt_w,
                   manuf_price_index = plt_G,
                   firms_real_profit = plt_eta,
                   manuf_real_wage_M = plt_omega) # make dataframe
    filename = format("{}_tau{}sigma{}mu{}R{}t{}sd{}", DateTime, tau, sig, mu, R, t, SD)
    newfilename = replace(filename, "."=>"p")
    CSV.write(newfilename * ".csv", df) # write into csv
end


In [ ]:
# run the simulation
for tau in [0.5, 0.7, 1.1, 1.7, 2.0, 2.6]
    params = [5.0, tau] # set parameters [sigma, tau]
    for itr in 1:5
        dualmigration_simulation(params)
    end
end
